In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler # conda install -c conda-forge imbalanced-learn

df_customer = pd.read_csv("./data/customer.csv")
df_category = pd.read_csv("./data/category.csv")
df_product = pd.read_csv("./data/product.csv")
df_receipt = pd.read_csv("./data/receipt.csv")
df_store = pd.read_csv("./data/store.csv")
df_geocode = pd.read_csv("./data/geocode.csv")

C:\Users\335395\AppData\Local\Continuum\anaconda3\envs\Yugo environment\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


P-041: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前日からの売上金額増減を計算せよ。なお、計算結果は10件表示すればよい。

In [2]:
df_receipt.head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90


In [3]:
df1 = df_receipt.groupby('sales_ymd').amount.sum().reset_index()
df1['difference'] = df1['amount'].diff(1)
df1.head(10)

,sales_ymd,amount,difference
0,20170101,33723,NaN
1,20170102,24165,-9558.0
2,20170103,27503,3338.0
3,20170104,36165,8662.0
4,20170105,37830,1665.0
5,20170106,32387,-5443.0
6,20170107,23415,-8972.0
7,20170108,24737,1322.0
8,20170109,26718,1981.0
9,20170110,20143,-6575.0


P-042: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、１日前、２日前、３日前のデータを結合せよ。結果は10件表示すればよい。

In [4]:
def ds_sum(df):
    return df.iloc[1]+df.iloc[2]+df.iloc[3]+df.iloc[4]

In [6]:
df2 = df_receipt.groupby('sales_ymd').amount.sum()
df2_1d = df2.shift(1)
df2_2d = df2.shift(2)
df2_3d = df2.shift(3)
df2_4d_sum = pd.concat([df2,df2_1d,df2_2d,df2_3d],axis=1).reset_index()
df2_4d_sum.columns = ['sales_ymd', 'amount_0d', 'amount_1d', 'amount_2d', 'amount_3d']
df2_4d_sum['4ds_sum'] = df2_4d_sum.apply(ds_sum,axis=1)
df2_4d_sum.head(10)

,sales_ymd,amount_0d,amount_1d,amount_2d,amount_3d,4ds_sum
0,20170101,33723,NaN,NaN,NaN,NaN
1,20170102,24165,33723.0,NaN,NaN,NaN
2,20170103,27503,24165.0,33723.0,NaN,NaN
3,20170104,36165,27503.0,24165.0,33723.0,121556.0
4,20170105,37830,36165.0,27503.0,24165.0,125663.0
5,20170106,32387,37830.0,36165.0,27503.0,133885.0
6,20170107,23415,32387.0,37830.0,36165.0,129797.0
7,20170108,24737,23415.0,32387.0,37830.0,118369.0
8,20170109,26718,24737.0,23415.0,32387.0,107257.0
9,20170110,20143,26718.0,24737.0,23415.0,95013.0


P-043： レシート明細データフレーム（df_receipt）と顧客データフレーム（df_customer）を結合し、性別（gender）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータフレーム（df_sales_summary）を作成せよ。性別は0が男性、1が女性、9が不明を表すものとする。

ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [7]:
df_receipt.head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90


In [8]:
df_customer.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2


In [9]:
df3 = pd.merge(df_receipt,df_customer,how='inner',on='customer_id')
df3['era'] = df3['age'].apply(lambda x: math.floor(x/10)*10)
df_sales_summary = pd.pivot_table(df3,index='era',columns='gender_cd',values='amount',aggfunc='sum')
df_sales_summary

gender_cd,0,1,9
era,,,
10,1591.0,149836.0,4317.0
20,72940.0,1363724.0,44328.0
30,177322.0,693047.0,50441.0
40,19355.0,9320791.0,483512.0
50,54320.0,6685192.0,342923.0
60,272469.0,987741.0,71418.0
70,13435.0,29764.0,2427.0
80,46360.0,262923.0,5111.0
90,NaN,6260.0,NaN


In [10]:
# age_list = df3_gen['age'].values.tolist() #listかseriesにしないとcut()に使えない
# labels = [ "{0} - {1}".format(i, i + 10) for i in range(0, 80, 10) ]#ラベル用
# c = pd.cut(age_list, np.arange(0, 81, 10),include_lowest=True, right=False,labels=labels)

# df3_gen_age = df3_gen['amount'].groupby(c).agg('sum').reset_index()
# df3_gen_age.head()

P-044： 前設問で作成した売上サマリデータフレーム（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータフレームから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を'00'、女性を'01'、不明を'99'とする。

In [11]:
df_sales_summary_T = df_sales_summary.stack().reset_index()
df_sales_summary_T.columns = ['era','gender_cd','amount']
df_sales_summary_T['gender_cd'] = df_sales_summary_T['gender_cd'].replace({0:'00',1:'01',9:'99'})
df_sales_summary_T

,era,gender_cd,amount
0,10,00,1591.0
1,10,01,149836.0
2,10,99,4317.0
3,20,00,72940.0
4,20,01,1363724.0
5,20,99,44328.0
6,30,00,177322.0
7,30,01,693047.0
8,30,99,50441.0
9,40,00,19355.0


P-045: 顧客データフレーム（df_customer）の生年月日（birth_day）は日付型（Date）でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [12]:
df_customer.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2


In [13]:
df_customer.dtypes

customer_id             object
customer_name           object
gender_cd                int64
gender                  object
birth_day               object
age                      int64
postal_cd               object
address                 object
application_store_cd    object
application_date         int64
status_cd               object
dtype: object

In [14]:
df5 = df_customer
df5['birth_day'] = pd.to_datetime(df5['birth_day'])

In [20]:
df5['birth_day'] = df5['birth_day'].dt.strftime('%Y%m%d')
df5.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS021313000114,大野 あや子,1,女性,19810429,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
1,CS037613000071,六角 雅彦,9,不明,19520401,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS031415000172,宇多田 貴美子,1,女性,19761004,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
3,CS028811000001,堀井 かおり,1,女性,19330327,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
4,CS001215000145,田崎 美紀,1,女性,19950329,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
